Get latest changes

In [1]:
%%bash
git pull

 
 
 
 

CalledProcessError: Command 'b'git pull\n'' returned non-zero exit status 1.

Import libraries

In [14]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils import data
import torchvision

import kaolin as kal
import open3d as o3d

import numpy as np
import math
from matplotlib import pyplot as plt
import time, multiprocessing, copy, datetime, random,os

In [15]:
from seeder import SEED, init_fn
from helpers import visualizer, logger, model_utils
from models import structure_transfer_models, texture_transfer_models
from models3d_dataset import Pix3D
import models.pointnet
from models.pointnet import Pointnet_Autoencoder
from defaults import DEFAULTS as D

Set Args here

In [16]:
n_points=1024
lr = 1e-4
n_epochs = 30
num_batch_chkpts = 5
num_epoch_chkpts = n_epochs
multiprocess=True

Setup dataset

In [17]:
dataset= Pix3D(n_points,categories=['chair'])
train_size, val_size, test_size = round(0.60 * dataset.__len__()),round(0.20 * dataset.__len__()),round(0.20 * dataset.__len__())
train_dataset, val_dataset, test_dataset = data.random_split(dataset,[train_size,val_size,test_size],
                                                            generator = torch.Generator().manual_seed(SEED))
n_workers = multiprocessing.cpu_count()//4 if multiprocess else 0
train_loader = data.DataLoader(train_dataset,batch_size=16,worker_init_fn=init_fn,num_workers=n_workers)
val_loader = data.DataLoader(val_dataset,batch_size=16,worker_init_fn=init_fn,num_workers=n_workers)
test_loader = data.DataLoader(test_dataset,batch_size=16,worker_init_fn=init_fn,num_workers=n_workers)

Setup models

In [18]:
model = Pointnet_Autoencoder(n_points=n_points).to(D.DEVICE())
img_feat_extractor = texture_transfer_models.VGG19()

Training algorithm

In [19]:
batch_train_chkpt = 1 if len(train_loader) <= num_batch_chkpts else len(train_loader)//num_batch_chkpts
batch_val_chkpt = 1 if len(val_loader) <= num_batch_chkpts else len(val_loader)//num_batch_chkpts
epoch_chkpt = 1 if n_epochs <= num_epoch_chkpts else n_epochs//num_epoch_chkpts

In [20]:
optimizer = torch.optim.Adam(model.parameters(),lr=lr)

In [21]:
since = int(round(time.time()*1000))
print('Start training')
for epoch in range(n_epochs):
    model.train()
    train_running_loss=0.0
    train_loss,val_loss=0.0,0.0
    for i, data_ in enumerate(train_loader,0):
        pointcloud,image = data_
        bs = pointcloud.shape[0]
        pointcloud = pointcloud.to(D.DEVICE())
        image = image.to(D.DEVICE())

        optimizer.zero_grad()
        image_feats = img_feat_extractor(image,layers=D.STYLE_LAYERS.get())

        output = model(pointcloud,image_feats)
        loss = models.pointnet.pointcloud_autoencoder_loss(output,pointcloud)
        loss.backward()
        optimizer.step()
        train_loss+= loss.item() * bs
        train_running_loss+=loss.item()
        if(i%batch_train_chkpt==batch_train_chkpt-1):
            print('[Epoch {} | Train Batch {}/{} ] Loss: {:.3f}'.format(epoch+1,i+1,len(train_loader),train_running_loss/batch_train_chkpt))
            train_running_loss=0.0
    model.eval()
    val_running_loss=0.0
    for j,data_ in enumerate(val_loader):
        pointcloud,image = data_
        bs = pointcloud.shape[0]
        pointcloud = pointcloud.to(D.DEVICE())
        image = image.to(D.DEVICE())
        with torch.no_grad():
            image_feats = img_feat_extractor(image,layers=D.STYLE_LAYERS.get())
            output = model(pointcloud,image_feats)
            loss,f_score,precision,recall = models.pointnet.pointcloud_autoencoder_loss(output,pointcloud,is_eval=True)
        val_loss+= loss.item() * bs
        val_running_loss+=loss.item()
        if(i%batch_val_chkpt==batch_val_chkpt-1):
            print('[Epoch {} | Val Batch {}/{} ] Loss: {:.3f}\t F-Score: {:.3f}\t Precision: {:.3f}\t Recall: {:.3f}'.format(epoch+1,j+1,
            len(val_loader),val_running_loss/batch_val_chkpt,f_score.mean(),precision.mean(),recall.mean()))
            val_running_loss=0.0
    print('\n[Epoch {}]\t Train Loss: {:.3f}\t  Validation Loss: {:.3f}\t \n'.format(epoch+1,
                                                                                train_loss/len(train_loader), 
                                                                                val_loss/len(val_loader)))
    if (epoch%epoch_chkpt==epoch_chkpt-1):
        gen_path = f'{model.__class__.__name__}_chkpt.pth'
        torch.save(model.state_dict(),gen_path)
        print(f'[Epoch {epoch+1}]\t Model saved in {gen_path}\n')
time_elapsed = int(round(time.time()*1000)) - since
print ('training time elapsed {}ms'.format(time_elapsed))
model_file = '{}_final.pth'.format(model.__class__.__name__)
gen_path = model_file
torch.save(model.state_dict(),gen_path)
print(f'Final Model saved in {gen_path}\n')

Start training
[Epoch 1 | Train Batch 1/3 ] Loss: 2.583
[Epoch 1 | Train Batch 2/3 ] Loss: 2.519
[Epoch 1 | Train Batch 3/3 ] Loss: 2.466
[Epoch 1 | Val Batch 1/1 ] Loss: 0.043	 F-Score: 0.108	 Precision: 0.093	 Recall: 0.128

[Epoch 1]	 Train Loss: 29.675	  Validation Loss: 0.429	 

[Epoch 1]	 Model saved in Pointnet_Autoencoder_epoch-1.pth

[Epoch 2 | Train Batch 1/3 ] Loss: 2.455
[Epoch 2 | Train Batch 2/3 ] Loss: 2.446
[Epoch 2 | Train Batch 3/3 ] Loss: 2.590
[Epoch 2 | Val Batch 1/1 ] Loss: 0.052	 F-Score: 0.096	 Precision: 0.081	 Recall: 0.117

[Epoch 2]	 Train Loss: 28.729	  Validation Loss: 0.521	 

[Epoch 2]	 Model saved in Pointnet_Autoencoder_epoch-2.pth

[Epoch 3 | Train Batch 1/3 ] Loss: 2.410
[Epoch 3 | Train Batch 2/3 ] Loss: 2.365
[Epoch 3 | Train Batch 3/3 ] Loss: 2.238
[Epoch 3 | Val Batch 1/1 ] Loss: 0.057	 F-Score: 0.080	 Precision: 0.069	 Recall: 0.098

[Epoch 3]	 Train Loss: 27.705	  Validation Loss: 0.570	 

[Epoch 3]	 Model saved in Pointnet_Autoencoder_epoch-3.

Visualize test pointclouds

In [22]:
pointclouds,images = iter(test_loader).next()
test_pointcloud = pointclouds[:10]
test_images = images[:10]
model.eval()
with torch.no_grad():
    image_feats = img_feat_extractor(test_images,layers=D.STYLE_LAYERS.get())
    final_output = model(test_pointcloud)

In [23]:
for tp, f in zip(test_pointcloud,final_output):
    visualizer.display_pointcloud(tp)
    visualizer.display_pointcloud(f)